**Read Me**

Our runtime uses [ GPU ]

This workbook uses two pre-trained models for language comprehension. One for english, one for not english. Using a Collab GPU enviornment optimizes performance. It is NOT possible to train both models in a single collab session. You will run out of memory and crash. 

You can load the weights using the code below. You can train one of the models in a collab session. Tips:
Each time you run a new version of the model, restart the environment.

*   Comment out/in the training of the model(s) you do not wish to train and load the data instead. 
*   Each time you run a new version of a model, restart and run all to avoid running out of memory. 
*   If you are tuning models, save the values locally in your collab session. This will improve performance verses trying to learn or load remotely. <br><br>

**Note:**
This notebook trains the **Multi-language** model, use the notebook link (at the bottom) to generate prediction/submission.



# **Import Libraries & Set Environment**

**Imports from Starter Notebook**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# stock imports from original notebook
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**Install Transformers & Sentencepiece**

These two libraries are used to load and process pretrained langauge models. 

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 89.2 MB/s 
     |████████████████████████████████| 3.3 MB 26.3 MB/s 
     |████████████████████████████████| 59 kB 8.5 MB/s 
     |████████████████████████████████| 895 kB 58.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install transformers[sentencepiece]

     |████████████████████████████████| 1.2 MB 5.0 MB/s 


**Load Dependencies**

Load dependencies from the installed transformers library along with tensorflow and sklearn. tqdm is used for progress bars. 

In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoModel, AutoTokenizer, XLMRobertaTokenizer, TFXLMRobertaModel
import sklearn
from sklearn.model_selection import train_test_split
from tqdm import tqdm

**Download Train & Test Data**

Load train and test data from github repo

In [ ]:
train = pd.read_csv("https://github.com/jeffreyssimon/Hello_Watson/blob/main/dataset/train.csv?raw=true") 
test = pd.read_csv("https://github.com/jeffreyssimon/Hello_Watson/blob/main/dataset/test.csv?raw=true")

## Data Transform Helper Functions

**Tokenize_Data**

Function to take a data frame, tokenizer, and process into (tokenized) x and y values.

In [ ]:
def tokenize_data(df, tokenizer, max_length):
    # tokenize
    text = df[['premise', 'hypothesis']].values.tolist()
    encoded = tokenizer.__call__(text, padding='max_length', max_length=max_length, truncation=True)
    # features
    x = encoded['input_ids']
    # labels
    y = None
    if 'label' in df.columns:
        y = df.label.values
    return x, y

**build_dataset**

Take x and y values and build them into a data set format which can be used to train or process a model

In [ ]:
def build_dataset(x, y, mode, batch_size):
  if mode == "train":
    dataset = (
        tf.data.Dataset
        .from_tensor_slices((x, y))
        .shuffle(2048)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE))
  elif mode == "valid":
    dataset = (
        tf.data.Dataset
        .from_tensor_slices((x, y))
        .batch(batch_size)
        .cache()
        .prefetch(tf.data.AUTOTUNE))
  elif mode == "test":
    dataset = (
        tf.data.Dataset
        .from_tensor_slices(x)
        .batch(batch_size))
  return dataset

# Tokenize & Format Data

**Define Pretrained Models**

One model for english language one for non english language. Both models are based on roberta-large with some extra nli training. Using an english only provides better results on english (which is half the data) than the multi language on english

In [ ]:
# Multi Language Model 
model_name_multi = 'joeddav/xlm-roberta-large-xnli'
# Max length and padding for embeddings
max_length = 120
# batch size for embeddings
batch_size = 16
# train test split
train_test = 0.2

**Split Train Data Set**

Splits the train data set into english and multi-language data sets. Optional split of data set into train and test sets for measuring model performance during training.

In [ ]:
# split data set into english and non english
tX_multi = train[train['language'] != 'English'].reset_index(drop=True)
# get train and test sets
tX_multi, val_multi = train_test_split(tX_multi, test_size=train_test)

# Set Parameters & Build Models

**Set Model Parameters**

Set parameters for the models. Note there are two models which share global parameters and have their own model specific parameters.

In [ ]:
# Global Parameters
RANDOM_STATE = 42
tf.random.set_seed(RANDOM_STATE)
# learning_rate = 1e-6
loss = 'sparse_categorical_crossentropy'
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-6,
    decay_steps=1000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
kernel_regularizer = tf.keras.regularizers.l1(l1=0.01)
# Multi Language Model Paremeters
model_name_multi = 'joeddav/xlm-roberta-large-xnli'
epochs_multi = 2

**Tokenize Multi-Language Dataset**

Tokenize the multi-language data and build a data set for processing through the multi language model.

In [ ]:
tokenizer_multi = AutoTokenizer.from_pretrained(model_name_multi)
tX_multi, tY_multi = tokenize_data(tX_multi, tokenizer_multi, max_length)
testX_multi, testY_multi = tokenize_data(val_multi, tokenizer_multi, max_length)

tX_multi = build_dataset(tX_multi, tY_multi, "train", batch_size)
testX_multi = build_dataset(testX_multi, testY_multi, "valid", batch_size)

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

**Build the Multi Language Model**

Build the model for assessing english language contradictions. Current model is an encoding layer, pooling layer, and output layer.

In [ ]:
input = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="input")

encoder = TFAutoModel.from_pretrained(model_name_multi)

encoder_out = encoder(input)[0]

pooling = tf.keras.layers.GlobalAveragePooling1D()(encoder_out)
# pooling = tf.keras.layers.GlobalMaxPooling1D()(encoder_out)
output = tf.keras.layers.Dense(3, activation='softmax', kernel_regularizer=kernel_regularizer)(pooling)

model_multi = tf.keras.models.Model(inputs=input, outputs=output)
model_multi.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Some layers from the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing TFXLMRobertaModel: ['classifier']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaModel were initialized from the model checkpoint at joeddav/xlm-roberta-large-xnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


**Train Multi-Language Model**

Train the model OR load the weights for the model.

In [ ]:
hist = model_multi.fit(tX_multi, epochs=epochs_multi, batch_size=32, validation_data=testX_multi, verbose=1)

Epoch 1/6
263/263 [==============================] - 237s 903ms/step - loss: 1.2771 - accuracy: 0.9624 - val_loss: 1.2447 - val_accuracy: 0.9743
Epoch 3/6
263/263 [==============================] - 237s 902ms/step - loss: 1.2355 - accuracy: 0.9764 - val_loss: 1.2312 - val_accuracy: 0.9762
Epoch 4/6
263/263 [==============================] - 237s 902ms/step - loss: 1.2170 - accuracy: 0.9788 - val_loss: 1.2221 - val_accuracy: 0.9790
Epoch 5/6
263/263 [==============================] - 237s 902ms/step - loss: 1.2051 - accuracy: 0.9833 - val_loss: 1.2189 - val_accuracy: 0.9800
Epoch 6/6
263/263 [==============================] - 237s 902ms/step - loss: 1.1857 - accuracy: 0.9890 - val_loss: 1.2168 - val_accuracy: 0.9781


In [ ]:
model_multi.save_weights('/content/gdrive/MyDrive/model/multi/')

# Prediction

Run this notebook for prediction and submission:
https://colab.research.google.com/drive/1zTK2tK817x8Zll9HumvmN_ZoeXhkmbBP#scrollTo=9o8D4eUIgQs5
